In [5]:
import math
import time

import cv2
import numpy as np
from scipy import ndimage
import src.utils as utils
from src.getDensity import getFilteredDensity
from src.tracker import Tracker
from datetime import datetime
import freud
from pynhhd import nHHD
import math
import scipy, scipy.fftpack


In [2]:
def poisson_reconstruct(grady, gradx, boundarysrc):
	# Thanks to Dr. Ramesh Raskar for providing the original matlab code from which this is derived
	# Dr. Raskar's version is available here: http://web.media.mit.edu/~raskar/photo/code.pdf

	# Laplacian
	gyy = grady[1:,:-1] - grady[:-1,:-1]
	gxx = gradx[:-1,1:] - gradx[:-1,:-1]
	f = np.zeros(boundarysrc.shape)
	f[:-1,1:] += gxx
	f[1:,:-1] += gyy

	# Boundary image
	boundary = boundarysrc.copy()
	boundary[1:-1,1:-1] = 0

	# Subtract boundary contribution
	f_bp = -4*boundary[1:-1,1:-1] + boundary[1:-1,2:] + boundary[1:-1,0:-2] + boundary[2:,1:-1] + boundary[0:-2,1:-1]
	f = f[1:-1,1:-1] - f_bp

	# Discrete Sine Transform
	tt = scipy.fftpack.dst(f, norm='ortho')
	fsin = scipy.fftpack.dst(tt.T, norm='ortho').T

	# Eigenvalues
	(x,y) = np.meshgrid(range(1,f.shape[1]+1), range(1,f.shape[0]+1), copy=True)
	denom = (2*np.cos(math.pi*x/(f.shape[1]+2))-2) + (2*np.cos(math.pi*y/(f.shape[0]+2)) - 2)

	f = fsin/denom

	# Inverse Discrete Sine Transform
	tt = scipy.fftpack.idst(f, norm='ortho')
	img_tt = scipy.fftpack.idst(tt.T, norm='ortho').T

	# New center + old boundary
	result = boundary
	result[1:-1,1:-1] = img_tt

	return result

In [ ]:
def match_depth(flow_x, flow_y):
    


In [2]:
def Young(flow, a, b, delta, E_1=2.2):
    
    w_1 = match_depth(flow[:,:,1],flow[:,:,0])
    
    E_2 = b*(w_1*E_1)/(a-w_1)+ delta


In [7]:
if __name__ == "__main__":
    tracker = Tracker(adaptive=True,
                      cuda=False)  # cuda=True if using opencv cuda
    #cap = cv2.VideoCapture(0, cv2.CAP_V4L2)
    top, bottom, left, right = (0, 720, 200, 980) # ROI
    """gstream_str = 'nvarguscamerasrc sensor-id=0 tnr-mode=2 ee-mode=2 aeantibanding=2 !'\
        ' video/x-raw(memory:NVMM), width=1280, height=720, format=(string)NV12, framerate=(fraction)60/1 !'\
            ' nvvidconv top={top} bottom={bottom} left={left} right={right} !'\
                ' video/x-raw, format=(string)GRAY8, width={width}, height={height}! appsink max-buffers=1 drop=true' \
                    .format(width=str(right-left), height=str(bottom-top), top=top, bottom=bottom, left=left, right=right)
    cap = cv2.VideoCapture(gstream_str, cv2.CAP_GSTREAMER)"""
    cap = cv2.VideoCapture(0)
    frame_num = 0.0
    start_time = time.time()
    capture = False
    count = 0
    while True: 
        success, img = cap.read(  )
        if capture:
            frame_num += 1.0
            img_cut = img[20:-20, 100:-100]

            # img = cv2.cvtColor(img, cv2.COLOR_YUV2GRAY_I420)
            img_gray = cv2.cvtColor(img_cut, cv2.COLOR_BGR2GRAY)
            # img = img[72:400, 175:480].copy()  
            # img = cv2.resize(img, (img.shape[1]//2,img.shape[0]//2))
            flow = tracker.track(img_gray) 

            boundarysrc = np.zeros_like(flow[:,:,0])
            """
            results = poisson_reconstruct(flow[:,:,1],flow[:,:,0],boundarysrc)
            
            Sn, St, d = utils.Helmholtz(flow)
            #rawdata_Sn.append(Sn)
            #rawdata_St.append(St)
  
            Fn = -0.714368+6.911549*Sn
            if Fn<0:
               Fn = 0
            print(Fn)
            """
            
            vis = utils.flowvis(img_cut,flow)
            density, threshold = getFilteredDensity(flow, use_cuda=True)
            density = (density + 1.0) / 1.0
            density = (density * 255.0).astype('uint8')
            #print("density max:",density.max(),"density min:", density.min())
            threshold, contact_boundary = utils.getContactBoundary(density)
            density[np.where(density < threshold)] = 0
            center_of_mass = ndimage.measurements.center_of_mass(density)
            density = cv2.applyColorMap(density, cv2.COLORMAP_HOT)
            cv2.circle(density,   
                       (int(center_of_mass[1]), int(center_of_mass[0])), 5,
                       (255, 0, 0), 2)
            #arrows = utils.put_optical_flow_arrows_on_image(
                #np.zeros_like(density), flow.download()[15:-15, 15:-15, :])
            arrows = utils.put_optical_flow_arrows_on_image(
                np.zeros_like(density), flow)
            #img_cut = img_cut[15:-15,15:-15]
            results = utils.show_multi_imgs(1, [img_cut, contact_boundary, arrows, density],(2,2))
            cv2.imshow('contact-edge',results)
            #print(time.time()-start_time)
            # cv2.imshow('img', img)
            # cv2.imshow('boundary', contact_boundary)
            # cv2.imshow('arrows', arrows)  
            # cv2.imshow('density', density)
        k = cv2.waitKey(1)
        if k & 0xFF == ord('q'):
            # cv2.imwrite('d3.jpg', img)
            break
        elif k & 0xFF == ord('s'):
             file_name = datetime.now().strftime("%m-%d-%H-%M-%S")
             print(file_name, " pic saved")
             cv2.imwrite(file_name+'.jpg', img_cut)
        elif k & 0xFF == ord('r'):
            tracker.reset()
            print("Tracker reset")
        elif k & 0xFF == ord('f'):
            file_name ='flow-'+str(count)
            print(file_name," flow saved")
            np.save("./flowdata/"+file_name+'.npy',flow)
            count += 1
        if time.time() - start_time > 1.0:
            capture = True
            fps = frame_num / (time.time() - start_time)

            print("{:.2f}".format(fps),
                  "FPS, current frame queue length: ",
                  tracker.getFrameQueueLength(),
                  end='\r')
    
            frame_num = 0.0
            start_time = time.time()
            
    cap.release()
    cv2.destroyAllWindows()

loop closed at frame  0e queue length:  2
loop closed at frame  0e queue length:  2
loop closed at frame  0e queue length:  1
loop closed at frame  0e queue length:  2
